In [149]:
# import libraries
import os
import time
import datetime
import pandas as pd
import glob
import sqlalchemy
import numpy as np

In [154]:
outdir='data/'

In [14]:
# read data
project_home = '/home/scube_dev/basic_analytic_data_platform/sample_project/'
dbpath = project_home+'/data/sg_weather.db'
engine = sqlalchemy.create_engine('sqlite:///'+dbpath)
df = pd.read_sql('raw_data', con=engine)

In [ ]:
# select data
# - use only last 5 year data
cur_date = datetime.datetime.now()
cutoff_date = datetime.datetime(cur_date.year-5, cur_date.month, cur_date.day).isoformat('T').split('T')[0]
df_sel = df[df['Datetime'] > cutoff_date]

In [136]:
# data labelling
category_names = {
    0: 'sunny',
    1: 'drizzle',
    2: 'rainy',
    3: 'stormy'
} # end name

def categorize(s):
    if s < 50:
        return 0
    elif 50 <= s < 200:
        return 1
    elif 200 <= s < 500:
        return 2
    else:
        return 3
    # end if
# end def

In [138]:
_df = df_sel[['Datetime', 'Daily Rainfall Total (mm)']].groupby('Datetime').agg(sum)
_df['Datetime'] = _df.index

cats = list(map(categorize, _df['Daily Rainfall Total (mm)']))
print({category_names[_]:cats.count(_) for _ in set(cats)})

_df['category'] = cats

In [141]:
# save to temporary file

In [155]:
outfilename = 'weather-data.label.%s.csv' % (str(int(time.time())),)
outfilename = outdir+'/'+outfilename

In [156]:
outfilename

'data//weather-data.label.1541248528.csv'

In [157]:
_df.to_csv(outfilename, index=None)